In [1]:
import xmlrpc.client
from xmlrpc.server import SimpleXMLRPCRequestHandler
from xmlrpc.server import SimpleXMLRPCServer
import time

In [2]:
server = xmlrpc.client.ServerProxy("http://localhost:8008")
stats_server = xmlrpc.client.ServerProxy("http://localhost:8018")

In [3]:
class ServerProxyForLab3:
    # обёртка над конкретным вызываемым методом конечного сервера для логирования
    class MethodCallLoggerDecorator:
        MAX_LOG_LENGTH = 35

        def __init__(self, xmlrpc_method, method_name: str):
                self.called_method = xmlrpc_method
                self.called_method_name = method_name

        def __call__(self, *args):
            print(f"Вызов из прокси-сервера функции '{self.called_method_name}'")

            start = time.time()
            print([*args])
            result = self.called_method(*args)
            elapsed_time_ms = (time.time() - start) * 1000

            print("Before add log")
            self.add_log(self.called_method_name, elapsed_time_ms, self.MAX_LOG_LENGTH)
            return result

        # Добавление в лог через прокси-сервер
        def add_log(self, method_name: str, elapsed_time_ms: float, max_log_length: int = 0) -> bool:
            result = False
            try:
                stats_server.add_log(method_name, elapsed_time_ms, max_log_length)
                result = True
            except ConnectionRefusedError:
                print('Stats server connection refused error')
            return result

    # для того, чтобы вызов функций выглядел так, будто они существуют у нашего proxy-сервера
    def _dispatch(self, name, params):
        logger_decorator = ServerProxyForLab3.MethodCallLoggerDecorator(server.__getattr__(name), name)
        return logger_decorator(*params)

    def __getattr__(self, name):
        return ServerProxyForLab3.MethodCallLoggerDecorator(server.__getattr__(name), name)


In [4]:
class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)
myproxyserver = SimpleXMLRPCServer(("localhost", 8028),
                                    requestHandler=RequestHandler,
                                    allow_none=True)
myproxyserver.register_introspection_functions()
myproxyserver.register_instance(ServerProxyForLab3())

In [5]:
myproxyserver.serve_forever()

Вызов из прокси-сервера функции 'ping'
[]
Before add log


127.0.0.1 - - [07/Nov/2023 19:08:59] "POST /RPC2 HTTP/1.1" 200 -


Вызов из прокси-сервера функции 'now'
[]
Before add log


127.0.0.1 - - [07/Nov/2023 19:09:02] "POST /RPC2 HTTP/1.1" 200 -


Вызов из прокси-сервера функции 'type'
[2]
Before add log


...

127.0.0.1 - - [07/Nov/2023 19:09:45] "POST /RPC2 HTTP/1.1" 200 -


Вызов из прокси-сервера функции 'threshold_binarization'
[<xmlrpc.client.Binary object at 0x00000227421512D0>, 70]
Before add log


127.0.0.1 - - [07/Nov/2023 19:09:49] "POST /RPC2 HTTP/1.1" 200 -


Вызов из прокси-сервера функции 'threshold_binarization'
[<xmlrpc.client.Binary object at 0x0000022741E88C90>, 70]


127.0.0.1 - - [07/Nov/2023 19:09:52] "POST /RPC2 HTTP/1.1" 200 -
